In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

# File Path
data = pd.read_excel(r"INSERT HERE")

# Columns
columns_of_interest = [f"X{i}" for i in range(1, 28)] + ["status", "industry"]
model_data = data[columns_of_interest]

# Prod & Serv 
production_industries = ['A', 'C', 'F']
service_industries = ['G', 'H', 'I', 'J', 'L', 'M', 'N', 'P', 'Q', 'R', 'S']

# Splitting Prod
production_data = model_data[model_data['industry'].isin(production_industries)]
X_prod = production_data.drop(['status', 'industry'], axis=1)
y_prod = production_data['status']
X_train_prod, X_test_prod, y_train_prod, y_test_prod = train_test_split(X_prod, y_prod, test_size=0.30, random_state=99, stratify=y_prod)

# Splitting Serv
service_data = model_data[model_data['industry'].isin(service_industries)]
X_serv = service_data.drop(['status', 'industry'], axis=1)
y_serv = service_data['status']
X_train_serv, X_test_serv, y_train_serv, y_test_serv = train_test_split(X_serv, y_serv, test_size=0.30, random_state=99, stratify=y_serv)

# DT prod
tree_model_prod = DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_leaf=2, min_samples_split=20, random_state=99)
tree_model_prod.fit(X_train_prod, y_train_prod)

# DT serv
tree_model_serv = DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_leaf=2, min_samples_split=20, random_state=99)
tree_model_serv.fit(X_train_serv, y_train_serv)

# Results
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]  
    accuracy = accuracy_score(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_proba)
    conf_matrix = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    return accuracy, auc_score, conf_matrix, class_report

# Results
industries = model_data['industry'].unique()
for industry in industries:
    industry_data = model_data[model_data['industry'] == industry]
    X_ind = industry_data.drop(['status', 'industry'], axis=1)
    y_ind = industry_data['status']
    X_train_ind, X_test_ind, y_train_ind, y_test_ind = train_test_split(X_ind, y_ind, test_size=0.30, random_state=99, stratify=y_ind)
    
    if industry in production_industries:
        model = tree_model_prod
    else:
        model = tree_model_serv

    accuracy_ind, auc_score_ind, conf_matrix_ind, class_report_ind = evaluate_model(model, X_test_ind, y_test_ind)
    
    #Print
    print(f"\nResults for Industry {industry}:")
    print("Accuracy:", accuracy_ind)
    print("AUC Score:", auc_score_ind)
